In [9]:
import datasets

# Load cambridge-climb/BabyLM dataset
dataset = datasets.load_dataset("cambridge-climb/BabyLM", "original_strict")

Generating train split: 10175723 examples [01:44, 97731.86 examples/s]
Generating validation split: 985958 examples [00:10, 97133.85 examples/s] 
Generating test split: 1008786 examples [00:10, 98204.10 examples/s]


In [17]:
df = dataset['train'].to_pandas()
df = df.drop(columns=['tagged_text'])
df['is_child'] = False
df['target_child_age'] = 0.2
df = df.rename(columns={'text': 'processed_gloss'})
df['language_code'] = 'en-us'

In [20]:
# Import phonemize
import sys
import os
os.environ['PHONEMIZER_ESPEAK_LIBRARY'] = '/opt/local/lib/libespeak-ng.dylib'
sys.path.append('../../')
from src.phonemize import phonemize_utterances, character_split_utterance

lines = df['processed_gloss'].tolist()
phonemized_lines = phonemize_utterances(lines)
df['phonemized_utterance'] = phonemized_lines
df['character_split_utterance'] = character_split_utterance(lines)

Phonemizing using language "EnglishNA"...
Using espeak backend with language code "en-us"...


In [22]:
num_empty = len(df[df['phonemized_utterance'] == ''])
print(f'WARNING: {num_empty} lines were not phonemized successfully. Dropping these.')
df = df[df['phonemized_utterance'] != '']
df['character_split_utterance'] = character_split_utterance(df['processed_gloss'].tolist())

In [23]:
df.to_csv('BabyLM-phonemized/train.csv', index=False)

In [29]:
df = dataset['validation'].to_pandas()
df = df.drop(columns=['tagged_text'])
df['is_child'] = False
df['target_child_age'] = 0.2
df = df.rename(columns={'text': 'processed_gloss'})
df['language_code'] = 'en-us'

lines = df['processed_gloss'].tolist()
phonemized_lines = phonemize_utterances(lines)
df['phonemized_utterance'] = phonemized_lines
df['character_split_utterance'] = character_split_utterance(lines)

num_empty = len(df[df['phonemized_utterance'] == ''])
print(f'WARNING: {num_empty} lines were not phonemized successfully. Dropping these.')
df = df[df['phonemized_utterance'] != '']
df['character_split_utterance'] = character_split_utterance(df['processed_gloss'].tolist())

# Subsamples the validation set to 1000 examples
df = df.sample(n=10000, random_state=42)

df.to_csv('BabyLM-phonemized/valid.csv', index=False)

Phonemizing using language "EnglishNA"...
Using espeak backend with language code "en-us"...


In [30]:
import datasets
from datasets import load_dataset

dataset_new = load_dataset('BabyLM-phonemized')

Generating train split: 7552848 examples [00:35, 211150.26 examples/s]
Generating valid split: 737303 examples [00:03, 211874.02 examples/s]
